<a href="https://colab.research.google.com/github/ssingh-rookie/2015/blob/master/NLP_imdb_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
imdb_data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9E2U9S/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9E2U9S/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9E2U9S/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
imdb_data

{'test': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int64)>}

In [0]:
train_data, test_data = imdb_data['train'], imdb_data['test']


In [0]:
import numpy as np

train_sentences = []
train_labels = []

for s,l in train_data:
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

test_sentences = []
test_labels = []

for s,l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

np_train_labels = np.array(train_labels)
np_test_labels = np.array(test_labels)

In [6]:
train_sentences[:5]

['b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."',
 "b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot devel

In [7]:
test_sentences[:5]

['b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel\'s absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven\'t laughed this hard since I saw THE FULL MONTY. (And, even then, I don\'t think I laughed quite this hard... So to speak.) Tukel\'s talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there\'s none of the over-the-top scenery chewing one might\'ve expected from a film like this). DING-A-LING-LESS is a film whose time has come."',
 'b"A blackly comic tale of a down-trodden prie

###global variables for tokens

In [0]:
vocab_size = 10000
oov_token = "<OOV>"
embedding_dim = 16
max_len = 120
trunc_type = 'post'

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

token = Tokenizer(num_words=vocab_size, oov_token=oov_token)
token.fit_on_texts(train_sentences)
word_index = token.word_index
reverse_word_index = token.index_word
train_sequences = token.texts_to_sequences(train_sentences)
train_pad = pad_sequences(train_sequences, maxlen=max_len, truncating=trunc_type)


In [0]:

test_sequences = token.texts_to_sequences(test_sentences)
test_pad = pad_sequences(test_sequences, maxlen=max_len)

In [60]:
test_pad.shape


(25000, 120)

In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_6 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 11526     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit(train_pad, np_train_labels, epochs=20, validation_data=(test_pad, np_test_labels))

Epoch 1/20
782/782 [==============================] - 4s 5ms/step - loss: 9.7813e-05 - accuracy: 1.0000 - val_loss: 0.8403 - val_accuracy: 0.8266
Epoch 2/20
782/782 [==============================] - 4s 5ms/step - loss: 6.1197e-05 - accuracy: 1.0000 - val_loss: 0.8770 - val_accuracy: 0.8263
Epoch 3/20
782/782 [==============================] - 4s 5ms/step - loss: 3.8300e-05 - accuracy: 1.0000 - val_loss: 0.9105 - val_accuracy: 0.8258
Epoch 4/20
782/782 [==============================] - 4s 5ms/step - loss: 2.4043e-05 - accuracy: 1.0000 - val_loss: 0.9452 - val_accuracy: 0.8260
Epoch 5/20
782/782 [==============================] - 4s 5ms/step - loss: 1.5279e-05 - accuracy: 1.0000 - val_loss: 0.9807 - val_accuracy: 0.8264
Epoch 6/20
782/782 [==============================] - 4s 5ms/step - loss: 9.7523e-06 - accuracy: 1.0000 - val_loss: 1.0142 - val_accuracy: 0.8258
Epoch 7/20
782/782 [==============================] - 4s 5ms/step - loss: 6.2205e-06 - accuracy: 1.0000 - val_loss: 1.0489 -

In [32]:
e = model.layers[0]
weights = e.get_weights()[0]
weights.shape

(10000, 16)

In [0]:
import io

file_meta = io.open("word_names.tsv", mode="w", encoding="utf-8")
file_vecs = io.open("word_vecs.tsv", mode="w", encoding="utf-8")

for i in range(1, vocab_size):
  word = reverse_word_index[i]
  vec = weights[i]
  tab_vec = "\t".join([str(x) for x in vec])
  file_meta.write(word + '\n')
  file_vecs.write(tab_vec + '\n')

file_meta.close()
file_vecs.close()

In [0]:
try:
  from google.colab import files
except:
  pass
else:
  files.download('meta.tsv')
  files.download('vecs.tsv')